In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fe53b184f367847aa55a2617e1b05248cac098b2c3dd14dae217169b3ce2e42b10bcc2af6d9b6787a4792a557f9705bb1eba86d761f2407d2'

In [2]:
import aocd
from aocd.models import Puzzle
day = 11
year = 2022
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [11]:
test_data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [125]:
data = puzzle.input_data.splitlines()
# data = test_data.splitlines()
len(data), data[:10]

(55,
 ['Monkey 0:',
  '  Starting items: 57',
  '  Operation: new = old * 13',
  '  Test: divisible by 11',
  '    If true: throw to monkey 3',
  '    If false: throw to monkey 2',
  '',
  'Monkey 1:',
  '  Starting items: 58, 93, 88, 81, 72, 73, 65',
  '  Operation: new = old + 2'])

In [118]:
import operator
import re
from collections import deque

class Monkey():
    def __init__(self, items=None, operation=None, test=None):
        self.items = items
        self.operation = operation
        self.test = test
        self.pass_to = []
        self.cnt = 0
        
def generate_operation(instruction):
    a, op, b = instruction.split(' ')
    if a != 'old':
        a = int(a)
    if b != 'old':
        b = int(b)
    op = {
        '+': operator.add,
        '*': operator.mul,
    }[op]
    
    def operation(x, a=a, b=b, op=op):
        if a == 'old':
            a = x
        if b == 'old':
            b = x
        return op(a, b)
    return operation

def get_test(val):
    return lambda x: (x%val)==0

def parse(inp):
    monkeys = []
    current = None
    product = 1
    for line in inp:
        line = line.lstrip()
        if line.startswith('Monkey'):
            if current:
                monkeys.append(current)
            current = Monkey()
            continue
        if line.startswith("Starting items"):
            items = line.split(':')[1]
            current.items = deque(map(int, items.split(',')))
        if line.startswith("Operation"):
            operation = line.split('= ')[1]
            current.operation = generate_operation(operation)
        if line.startswith("Test"):
            div = int(re.findall('\d+', line).pop())
            product *= div
            current.test =  get_test(div)
        if line.startswith("If"):
            current.pass_to.append(int(re.findall('\d+', line).pop()))
    if current:
        monkeys.append(current)
    return monkeys, product

def play(monkeys, nrounds=20, part=1):
    for r in range(nrounds):
        for i, monkey in enumerate(monkeys):
            while monkey.items:
                item = monkey.items.popleft()
                monkey.cnt += 1
                item = monkey.operation(item)
                if part == 1:
                    item = item // 3
                if monkey.test(item):
                    monkeys[monkey.pass_to[0]].items.append(item)
                else:
                    monkeys[monkey.pass_to[1]].items.append(item)
                    
def playB(monkeys, nrounds=20, prod=1):
    for r in range(nrounds):
        for i, monkey in enumerate(monkeys):
            while monkey.items:
                item = monkey.items.popleft()
                monkey.cnt += 1
                item = monkey.operation(item)
                item = item % prod
                if monkey.test(item):
                    monkeys[monkey.pass_to[0]].items.append(item)
                else:
                    monkeys[monkey.pass_to[1]].items.append(item)

In [119]:
monkeys, _ = parse(data)
play(monkeys, 20)
[m.cnt for m in monkeys]

[101, 95, 7, 105]

In [120]:
sorted([m.cnt for m in monkeys])[-2:]

[101, 105]

In [96]:
puzzle.answer_a = operator.mul(*sorted([m.cnt for m in monkeys])[-2:])

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


# Part 2

In [126]:
%%time
monkeys, prod = parse(data)
playB(monkeys, 10000, prod=prod)
[m.cnt for m in monkeys]

CPU times: user 568 ms, sys: 3.67 ms, total: 572 ms
Wall time: 569 ms


[165766, 161105, 159985, 9209, 18375, 166922, 13590, 169205]

In [127]:
sorted([m.cnt for m in monkeys])[-2:]

[166922, 169205]

In [128]:
puzzle.answer_b = operator.mul(*sorted([m.cnt for m in monkeys])[-2:])

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 11! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
